In [2]:
import os

import schematics as scm
import schematics.types as scmt

import pandas as pd
import numpy as np

%cd ~/flatiron/python
import flatiron.core.tools as fict
import flatiron.core.logging as ficl
import flatiron.models.unet as fimu
from flatiron.core.pipeline import *

/home/ubuntu/flatiron/python


In [3]:
class FooConfig(scm.Model):
    bar = scmt.StringType(required=True)

class FooPipeline(PipelineBase):
    @property
    def model_config(self):
        return FooConfig
    
    def build(self):
        pass

def create_fake_asset(root):
    chunks = [
        'data/chunk_f00.npy',
        'data/chunk_f01.npy',
    ]
    os.makedirs(root, exist_ok=True)
    os.makedirs(Path(root, 'data'), exist_ok=True)
    data = pd.DataFrame()
    data['filepath_relative'] = chunks
    data['asset_path'] = root
    csv_path = Path(root, 'info.csv').as_posix()
    data.to_csv(csv_path, index=False)
    shape = (100, 10, 10, 3)
    arr = np.ones(shape, dtype=np.float16)
    for chunk in chunks:
        np.save(Path(root, chunk), arr)
    return data

In [4]:
root = '/tmp/ajshgdjhasd'
d = create_fake_asset(root)


config = f'''
model:
    bar: baz
dataset:
    source: {root}
    load_limit: 100
    split_index: 3
optimizer: {{}}
compile:
    loss: jaccard_loss
    metrics:
        - jaccard
        - dice
callbacks:
    project: test
    root: {root}
    timezone: 'America/Los_Angeles'
fit:
    batch_size: 32
logger:
    timezone: 'America/Los_Angeles'
    slack_url: 'https://hooks.slack.com/services/{uri}'
    slack_channel: dev
'''

p = FooPipeline \
    .from_string(config) \
    .load() \
    .train_test_split() \
    .unload()

May not total to 100% - Loading Dataset Chunks:   0%|          | 0/2 [00:00<?, ?it/s]


RUN TIME:
```0.02 seconds (0:00:00.017952)```
POST TIME:
```2023-02-01T08:00:53.735750-08:00```
CONFIG:
```dataset:
    load_limit: 100
    load_shuffle: false
    source: /tmp/ajshgdjhasd
    split_axis: -1
    split_index: 3
    split_random_state: 42
    split_shuffle: true
    split_test_size: 0.2
    split_train_size: null
```
   
